In [ ]:
!pip install openai==0.28.0



In [ ]:
import openai
import json
import re
import os
import time
from tqdm import tqdm

# ------------------------------------------------------------------
# Use your provided API key:
# ------------------------------------------------------------------
openai.api_key = "API KEY"
# ------------------------------------------------------------------

def extract_json(json_string: str) -> str:
    """
    Extract JSON from a string that may contain triple backticks.
    If we find fenced code (```json ... ```), we remove it.
    Otherwise, we return the original string stripped.
    """
    pattern = r"```(?:json)?(.*?)```"
    match = re.search(pattern, json_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return json_string.strip()

def augment_queries_for_video(video_id, queries, max_retries=3, backoff=5):
    """
    Sends all queries for one video in a single request.
    Returns a list of augmented queries in the same order.
    Retries up to 'max_retries' times if a RateLimitError or other OpenAIError occurs.
    """
    # Build a single user message that includes all queries:
    user_content = (
        "You will be given a list of sentences. "
        "For each sentence, generate a significantly longer version, "
        "while keeping the original meaning.\n\n"
        "IMPORTANT: Return only valid JSON (no triple backticks) with the augmented sentences in a JSON array.\n\n"
        "Here is the list of sentences:\n"
    )
    for i, q in enumerate(queries, start=1):
        user_content += f"{i}. {q}\n"

    # Attempt up to max_retries
    for attempt in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "You are a helpful assistant that processes multiple queries at once. "
                            "You will receive a list of sentences and must return a JSON array of augmented sentences. "
                            "Do not include triple backticks or any extra text—only return valid JSON."
                        )
                    },
                    {
                        "role": "user",
                        "content": user_content
                    }
                ],
                temperature=0.7
            )

            completion_text = response["choices"][0]["message"]["content"]
            cleaned_text = extract_json(completion_text)

            # Attempt to parse as JSON
            augmented_list = json.loads(cleaned_text)
            if not isinstance(augmented_list, list):
                raise ValueError("Response was not a JSON list.")

            return augmented_list

        except openai.error.RateLimitError as e:
            print(f"[RateLimitError] {e}. Retrying in {backoff} seconds...")
            time.sleep(backoff)
        except openai.error.OpenAIError as e:
            print(f"[OpenAIError] {e}. Retrying in {backoff} seconds...")
            time.sleep(backoff)
        except (ValueError, json.JSONDecodeError) as e:
            # If the JSON is invalid or we can't parse it at all, we won't retry infinitely
            # We'll just raise the error so we can stop for this video.
            raise ValueError(
                f"[JSONDecodeError] Could not parse model output for video '{video_id}'. Output was:\n{completion_text}"
            ) from e
        except Exception as e:
            print(f"[Unexpected Error] {e}")
            return None

    # If all retries fail (e.g., repeated RateLimitError), return None
    return None

def augment_json_file(input_json_path, output_json_path):
    """
    Reads an input JSON with format:
        {
            "video_file_1": ["query1", "query2", ...],
            "video_file_2": ["query1", "query2", ...],
            ...
        }
    For each video, it sends all queries at once to the API, expecting a JSON list in return.

    Partial results are saved to output_json_path, so if an error occurs,
    you won't lose progress on previously augmented videos.

    Additionally, uses a tqdm progress bar to show how many videos remain.
    """
    # Load the original data
    with open(input_json_path, "r", encoding="utf-8") as f:
        original_data = json.load(f)

    # If an output file already exists, load partial data
    if os.path.exists(output_json_path):
        with open(output_json_path, "r", encoding="utf-8") as f:
            augmented_data = json.load(f)
    else:
        augmented_data = {}

    all_videos = list(original_data.keys())
    total_videos = len(all_videos)

    print(f"Total number of videos to process: {total_videos}")

    # Use tqdm progress bar
    with tqdm(total=total_videos, desc="Processing videos") as pbar:
        for video_id in all_videos:
            # Check if this video is already done
            if video_id in augmented_data:
                pbar.update(1)
                continue  # Skip to the next

            print(f"\nNow augmenting queries for '{video_id}'")
            queries = original_data[video_id]

            # Call the augmentation function
            try:
                augmented_queries = augment_queries_for_video(video_id, queries)
            except ValueError as e:
                # If JSON parsing error or invalid response, we stop here
                # so the script eventually re-runs from the beginning.
                print(e)
                print(f"Failed to augment queries for '{video_id}'. Saving partial data and stopping.")
                break

            if augmented_queries is None:
                # None means we tried max times and still got an error
                print(f"Failed to augment queries for '{video_id}' after multiple retries. Saving partial data and stopping.")
                break

            # If we reach here, we have valid augmented queries
            augmented_data[video_id] = augmented_queries

            # Save partial results after each video
            with open(output_json_path, "w", encoding="utf-8") as f:
                json.dump(augmented_data, f, ensure_ascii=False, indent=4)

            # Update the progress bar
            pbar.update(1)

    print("\nAugmentation process complete.")
    print(f"Final augmented data saved to: {output_json_path}")


# ----------------------------------------------------------------------------
# Example usage: the script re-runs itself if any error occurs
# ----------------------------------------------------------------------------
if __name__ == "__main__":
    input_file_path = "/content/MSRVTT_captions_test.json"
    output_file_path = "/content/output_MSRVTT.json"

    while True:
        try:
            augment_json_file(input_file_path, output_file_path)
            # If augment_json_file completes with no fatal error, break the loop
            break
        except Exception as e:
            # If there's ANY error not caught inside augment_json_file,
            # we simply print and re-run
            print(f"\n[RE-RUN TRIGGER] An error occurred: {e}")
            print("Re-running the entire script...\n")
            time.sleep(2)
            continue


Total number of videos to process: 1000


Processing videos:   0%|          | 0/1000 [00:00<?, ?it/s]


Now augmenting queries for 'video9032'


Processing videos:  79%|███████▉  | 788/1000 [00:12<00:03, 65.48it/s]


Now augmenting queries for 'video7169'

Now augmenting queries for 'video9818'

Now augmenting queries for 'video8089'


...


Now augmenting queries for 'video7112'


Processing videos: 100%|█████████▉| 999/1000 [17:28<00:04,  4.32s/it]


Now augmenting queries for 'video8978'


Processing videos: 100%|██████████| 1000/1000 [17:33<00:00,  1.05s/it]


Augmentation process complete.
Final augmented data saved to: /content/output_MSRVTT.json


In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define file paths
colab_json_path = "/content/output_MSRVTT.json"  # Existing file in Colab
drive_json_path = "/content/drive/My Drive/output_MSRVTT.json"  # Destination in Drive

# Step 3: Check if the file exists in Colab
if os.path.exists(colab_json_path):
    # Copy the file from Colab to Google Drive
    shutil.copy(colab_json_path, drive_json_path)
    print(f"File successfully saved to Google Drive at: {drive_json_path}")
else:
    print(f"Error: File not found at {colab_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File successfully saved to Google Drive at: /content/drive/My Drive/output_MSRVTT.json
